In [1]:
#Imports

import os
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

rc('animation', html='jshtml')

In [5]:
#Get the data. I renamed the parent file folder to "data" for simplicity. Also get the supplementary data. I won't change that file though so we retain
#information about if it was flipped, field position, etc.
DATA_DIR = "/Users/ameyavelankar/Library/CloudStorage/OneDrive-UW/Writing/NFLBIGDATA/data/"

input_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/input_2023_w*.csv")))
output_files = sorted(glob.glob(os.path.join(DATA_DIR, "train/output_2023_w*.csv")))

df_in = pd.concat((pd.read_csv(p) for p in tqdm(input_files, desc="loading inputs")), ignore_index=True)
df_out = pd.concat((pd.read_csv(p) for p in tqdm(output_files, desc="loading outputs")), ignore_index=True)
sup_file=pd.read_csv(os.path.join(DATA_DIR, "supplementary_data.csv"))




loading inputs:   0%|          | 0/18 [00:00<?, ?it/s]

loading outputs:   0%|          | 0/18 [00:00<?, ?it/s]

/var/folders/y_/lhp104kx7f177lpvvd4rn5pw0000gn/T/ipykernel_33790/1525282006.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  sup_file=pd.read_csv(os.path.join(DATA_DIR, "supplementary_data.csv"))


In [6]:
# Generate mapping dictionaries from df_in
# now we have a single index for each game and play from 0->n

game_map_in = {gid: i for i, gid in enumerate(sorted(df_in['game_id'].unique()))}
play_maps_in = {}
for gid in df_in['game_id'].unique():
    plays = sorted(df_in[df_in['game_id'] == gid]['play_id'].unique())
    play_maps_in[gid] = {pid: i for i, pid in enumerate(plays)}

# Now you can use these for mapping or creating the mapping DataFrame
mapping_rows = []
flat_play_counter = 0
flat_play_map = {}  # (game_id, play_id) -> flat_play_id
for gid_orig, gid_new in game_map_in.items():
    for pid_orig, pid_new in play_maps_in[gid_orig].items():
        mapping_rows.append({
            'game_id': gid_orig,
            'play_id': pid_orig,
            'game_id_new': gid_new,
            'play_id_new': pid_new,
            'flat_play_id': flat_play_counter
        })
        flat_play_map[(gid_orig, pid_orig)] = flat_play_counter
        flat_play_counter += 1

mapping_df = pd.DataFrame(mapping_rows)
#print("Mapping DataFrame (original <-> normalized labels):")
display(mapping_df.head(10))
#print(flat_play_map)
# flat_play_map: (game_id, play_id) -> flat_play_id
# mapping_df['flat_play_id']: unique integer for each play in the dataset


,game_id,play_id,game_id_new,play_id_new,flat_play_id
0,2023090700,101,0,0,0
1,2023090700,194,0,1,1
2,2023090700,219,0,2,2
3,2023090700,361,0,3,3
4,2023090700,436,0,4,4
5,2023090700,461,0,5,5
6,2023090700,530,0,6,6
7,2023090700,621,0,7,7
8,2023090700,713,0,8,8
9,2023090700,736,0,9,9


In [7]:
# Add flat_play_id to df_in, df_out, and supplementary

def add_flat_play_id(df, flat_play_map):
    if 'game_id' in df.columns and 'play_id' in df.columns:
        df['flat_play_id'] = [flat_play_map.get((gid, pid), -1) for gid, pid in zip(df['game_id'], df['play_id'])]
    return df

df_in = add_flat_play_id(df_in, flat_play_map)
df_out = add_flat_play_id(df_out, flat_play_map)
sup_file = add_flat_play_id(sup_file, flat_play_map)


In [8]:
#Now we normalize so every play is going "left" by flipping the coordinates for plays going "left" (weird naming convention from the data source)


# find unique flat_play_id values for plays with play_direction == "left"
left_flat_play_ids = np.sort(df_in.loc[df_in['play_direction'] == "left", "flat_play_id"].unique())
print(f"Found {len(left_flat_play_ids)} unique flat_play_id with play_direction == 'left'")

# reverse x, y, and dir for all rows in df_in that belong to plays going "right"
mask = df_in['flat_play_id'].isin(left_flat_play_ids)
n_rows = mask.sum()
print(f"Reversing x,y,dir for {n_rows} rows (plays with play_direction == 'right')")

# horizontal flip (field length 0..120), vertical flip (0..53.3), and reverse heading (degrees)
if 'x' in df_in.columns:
    df_in.loc[mask, 'x'] = 120.0 - df_in.loc[mask, 'x']
if 'dir' in df_in.columns:
    df_in.loc[mask, 'dir'] = (360-df_in.loc[mask, 'dir'] ) % 360.0
if 'o' in df_in.columns:
    df_in.loc[mask, 'o'] = (360-df_in.loc[mask, 'o']) % 360.0
if 'ball_land_x' in df_in.columns:
    df_in.loc[mask, 'ball_land_x'] = 120.0 - df_in.loc[mask, 'ball_land_x']

# apply the same flip to df_out using the flat_play_ids for left-directed plays
mask_out = df_out['flat_play_id'].isin(left_flat_play_ids)

n_rows_out = mask_out.sum()
print(f"Reversing x,y,dir for {n_rows_out} rows in df_out (plays with play_direction == 'left')")

if 'x' in df_out.columns:
    df_out.loc[mask_out, 'x'] = 120.0 - df_out.loc[mask_out, 'x']


Found 6991 unique flat_play_id with play_direction == 'left'
Reversing x,y,dir for 2421505 rows (plays with play_direction == 'right')
Reversing x,y,dir for 280327 rows in df_out (plays with play_direction == 'left')


In [9]:
#Now I also want the ball to always land in the lower half of the field (y<26.65)

# find unique flat_play_id values for plays where ball_land_y >= 26.65
high_ball_land_flat_play_ids = np.sort(df_in.loc[df_in['ball_land_y'] >= 26.65, 'flat_play_id'].unique())
print(f"Found {len(high_ball_land_flat_play_ids)} unique flat_play_id with ball_land_y >= 26.65")

# reverse y for all rows in df_in that belong to plays where ball_land_y >= 26.65
mask_ball_land = df_in['flat_play_id'].isin(high_ball_land_flat_play_ids)
n_rows_ball_land = mask_ball_land.sum()

display(df_in.loc[mask_ball_land].head())
print(f"Reversing y for {n_rows_ball_land} rows (plays with ball_land_y >= 26.65)")
if 'y' in df_in.columns:
    df_in.loc[mask_ball_land, 'y'] = 53.3 - df_in.loc[mask_ball_land, 'y']
if 'dir' in df_in.columns:
    df_in.loc[mask_ball_land, 'dir'] = (180 - df_in.loc[mask_ball_land, 'dir']) % 360
if 'o' in df_in.columns:
    df_in.loc[mask_ball_land, 'o'] = (180 - df_in.loc[mask_ball_land, 'o']) % 360
if 'ball_land_y' in df_in.columns:
    df_in.loc[mask_ball_land, 'ball_land_y'] = 53.3 - df_in.loc[mask_ball_land, 'ball_land_y']

display(df_in.loc[mask_ball_land].head())
#Now for df_out
mask_ball_land_out = df_out['flat_play_id'].isin(high_ball_land_flat_play_ids)
n_rows_ball_land_out = mask_ball_land_out.sum()
print(f"Reversing y for {n_rows_ball_land_out} rows in df_out (plays with ball_land_y >= 26.65)")
if 'y' in df_out.columns:
    df_out.loc[mask_ball_land_out, 'y'] = 53.3 - df_out.loc[mask_ball_land_out, 'y']

Found 7019 unique flat_play_id with ball_land_y >= 26.65


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,flat_play_id
871,2023090700,361,False,54527,1,right,22,Bryan Cook,6-1,210,...,35.36,26.64,0.35,0.50,82.06,276.02,16,26.1,49.18,3
872,2023090700,361,False,54527,2,right,22,Bryan Cook,6-1,210,...,35.40,26.66,0.41,0.73,69.00,276.02,16,26.1,49.18,3
873,2023090700,361,False,54527,3,right,22,Bryan Cook,6-1,210,...,35.44,26.68,0.47,0.82,56.53,274.27,16,26.1,49.18,3
874,2023090700,361,False,54527,4,right,22,Bryan Cook,6-1,210,...,35.48,26.72,0.62,1.09,44.42,273.57,16,26.1,49.18,3
875,2023090700,361,False,54527,5,right,22,Bryan Cook,6-1,210,...,35.52,26.79,0.80,1.27,34.29,273.57,16,26.1,49.18,3


Reversing y for 2426770 rows (plays with ball_land_y >= 26.65)


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,flat_play_id
871,2023090700,361,False,54527,1,right,22,Bryan Cook,6-1,210,...,35.36,26.66,0.35,0.50,97.94,263.98,16,26.1,4.12,3
872,2023090700,361,False,54527,2,right,22,Bryan Cook,6-1,210,...,35.40,26.64,0.41,0.73,111.00,263.98,16,26.1,4.12,3
873,2023090700,361,False,54527,3,right,22,Bryan Cook,6-1,210,...,35.44,26.62,0.47,0.82,123.47,265.73,16,26.1,4.12,3
874,2023090700,361,False,54527,4,right,22,Bryan Cook,6-1,210,...,35.48,26.58,0.62,1.09,135.58,266.43,16,26.1,4.12,3
875,2023090700,361,False,54527,5,right,22,Bryan Cook,6-1,210,...,35.52,26.51,0.80,1.27,145.71,266.43,16,26.1,4.12,3


Reversing y for 280072 rows in df_out (plays with ball_land_y >= 26.65)


In [10]:
# Filter sup_file to only plays present in df_in using flat_play_id
valid_flat_play_ids = set(df_in['flat_play_id'].unique())
sup_file_filtered = sup_file[sup_file['flat_play_id'].isin(valid_flat_play_ids)].copy()
print(f"Filtered supplementary data: {sup_file_filtered.shape[0]} rows (out of {sup_file.shape[0]})")

Filtered supplementary data: 14108 rows (out of 18009)


In [ ]:
# Save the processed dataframes to disk
import os

output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

# Save as parquet for fast loading
in_path = os.path.join(output_dir, 'df_in.parquet')
out_path = os.path.join(output_dir, 'df_out.parquet')
sup_path = os.path.join(output_dir, 'sup_file.parquet')

print(f"Saving df_in to {in_path}")
df_in.to_parquet(in_path, index=False)
print(f"Saving df_out to {out_path}")
df_out.to_parquet(out_path, index=False)
print(f"Saving sup_file to {sup_path}")
sup_file.to_parquet(sup_path, index=False)


In [ ]:
# Save the mapping DataFrame as parquet
mapping_path = os.path.join(output_dir, 'mapping_df.parquet')
print(f"Saving mapping_df to {mapping_path}")
mapping_df.to_parquet(mapping_path, index=False)


In [ ]:

sup_csv_path = os.path.join(output_dir, 'sup_file.csv')
sup_file.to_csv(sup_csv_path, index=False)
print(f"Saved sup_file to {sup_csv_path}")

sup_filtered_csv_path = os.path.join(output_dir, 'sup_file_filtered.csv')
sup_file_filtered.to_csv(sup_filtered_csv_path, index=False)
print(f"Saved sup_file_filtered to {sup_filtered_csv_path}")